In [1]:
!pip install requests
!pip install beautifulsoup4

In [2]:
import numpy as np
import pandas as pd
import requests
import bs4
import lxml.etree as xml

In [31]:
base_url = 'https://www.tripadvisor.co.id'
urls = [
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d14000352-Reviews-The_Zuri_Palembang-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d2526653-Reviews-The_Arista_Hotel_Palembang-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d14025300-Reviews-Wyndham_Opi_Hotel_Palembang-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d12215506-Reviews-The_Excelton_Hotel-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d2513565-Reviews-Sintesa_Peninsula_Hotel_Palembang-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d1953135-Reviews-The_Alts_Hotel-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d486595-Reviews-Novotel_Palembang_Hotel_Residence-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d2177141-Reviews-Aston_Palembang_Hotel_Conference_Centre-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d11810067-Reviews-THE_1O1_Palembang_Rajawali-Palembang_South_Sumatra_Sumatra.html',
  'https://www.tripadvisor.co.id/Hotel_Review-g608501-d1493995-Reviews-Grand_Inna_Daira_Palembang-Palembang_South_Sumatra_Sumatra.html '
];

In [34]:
def scrape_reviews(url, num_page):
  print(f'Num page: {num_page}')
  
  reviewers = np.array([])
  review_titles = np.array([])
  reviews = np.array([])
  ratings = np.array([])
  dates = np.array([])

  page = bs4.BeautifulSoup(requests.get(url, {}).text, 'lxml')
  reviews_dom = page.find_all('div', { 'class': 'hotels-community-tab-common-Card__card--ihfZB hotels-community-tab-common-Card__section--4r93H' })
  for dom in reviews_dom:
    reviewer = dom.find('a', { 'class': 'ui_header_link' }).text
    review_title = dom.find('a', { 'class': 'location-review-review-list-parts-ReviewTitle__reviewTitleText--2tFRT' }).span.span.text
    review = dom.find('q', { 'class': 'location-review-review-list-parts-ExpandableReview__reviewText--gOmRC' }).span.text
    rating = dom.find('span', { 'class': 'ui_bubble_rating' }).attrs['class'][1].split('_')[1]
    date = dom.find('div', { 'class': 'social-member-event-MemberEventOnObjectBlock__event_type--3njyv' }).contents[0].text.split('ulasan')[1].lstrip()

    reviewers = np.append(reviewers, reviewer)
    review_titles = np.append(review_titles, review_title)
    reviews = np.append(reviews, review)
    ratings = np.append(ratings, rating)
    dates = np.append(dates, date)

  df = pd.DataFrame({
    'reviewer': reviewers,
    'review_title': review_titles,
    'reviews': reviews,
    'rating': ratings,
    'date': dates
  })

  current_page = page.find('span', { 'class': 'pageNum' })
  if current_page:
    next = current_page.find_next('a', href=True)
    if next and next['href']:
      num_page += 1
      df = df.append(scrape_reviews(base_url + next['href'], num_page), ignore_index = True)

  return df



df = pd.DataFrame({
    'reviewer': [],
    'review_title': [],
    'reviews': [],
    'rating': [],
    'date': []
})
for url in urls:
  print(url)
  df = df.append(scrape_reviews(url, 1), ignore_index=True)

https://www.tripadvisor.co.id/Hotel_Review-g608501-d14000352-Reviews-The_Zuri_Palembang-Palembang_South_Sumatra_Sumatra.html
Num page: 1
Num page: 2
Num page: 3
Num page: 4
Num page: 5
Num page: 6
Num page: 7
Num page: 8
Num page: 9
Num page: 10
Num page: 11
Num page: 12
Num page: 13
Num page: 14
Num page: 15
Num page: 16
Num page: 17
Num page: 18
Num page: 19
Num page: 20
Num page: 21
Num page: 22
Num page: 23
Num page: 24
Num page: 25
Num page: 26
Num page: 27
Num page: 28
Num page: 29
Num page: 30
Num page: 31
Num page: 32
Num page: 33
Num page: 34
Num page: 35
Num page: 36
https://www.tripadvisor.co.id/Hotel_Review-g608501-d2526653-Reviews-The_Arista_Hotel_Palembang-Palembang_South_Sumatra_Sumatra.html
Num page: 1
Num page: 2
Num page: 3
Num page: 4
Num page: 5
Num page: 6
Num page: 7
Num page: 8
Num page: 9
Num page: 10
Num page: 11
Num page: 12
Num page: 13
Num page: 14
Num page: 15
Num page: 16
Num page: 17
Num page: 18
Num page: 19
Num page: 20
Num page: 21
Num page: 22
Num pag

In [35]:
df.head()

,reviewer,review_title,reviews,rating,date
0,Harie,Cozy hotel dgn budget terjangku,"Hotel nyaman, keluatan mewah, sarapan enak, cu...",40,Apr 2020
1,Padli,The zuri Palembang the best,The zuri terbaik di kota Palembang apalagi wai...,50,Mar 2020
2,Bayu pranata,Pertama menginap dan Akan kembali terus ke the...,Pertama kali menginap januari tanggal 5 saya p...,50,Feb 2020
3,arifirmansyah,Di kasih suprise,Waw.. kaget tiba2 dikasih kue ultah waktu sara...,50,Feb 2020
4,Vivi yuriyanti,Thx to the zuri hotel,"Makanan banyak variasi, fresh dan enak Service...",50,Feb 2020


In [36]:
df.to_csv('reviews.csv')